#### Enunciado

Imagine uma base de dados de uma universidade que contenha as seguintes tabelas e seus respectivos campos organizados da seguinte forma:

tabela_alunos → colunas = (ra, nome, endereco, cidade);

tabela_disciplinas → colunas = (cod_disc, nome_disc, carga_hor)

tabela_professores → colunas = (cod_prof, nome, endereco, cidade)

tabela_turma → colunas = (cod_disc, cod_turma, cod_prof, ano, semestre)

tabela_historico, colunas = (ra, cod_disc, cod_turma, cod_prof, ano, semestre, frequencia, nota)

a) Monte uma consulta que informe quantos alunos cursaram a disciplina de Banco de Dados em 2019 e 2020;

b) Monte uma consulta que exiba os nomes dos alunos que não foram reprovados em nenhuma disciplina (considere nota mínima igual a 6 para aprovação).


#### Importar bibliotecas

In [2]:
import pandas as pd

import numpy as np

import sqlite3

import sqlalchemy as sa

from sqlalchemy import create_engine

#### Criar as tabelas usando pandas

In [6]:
# tabela_alunos → colunas = (ra, nome, endereco, cidade);

df_alunos = pd.DataFrame(columns = ['ra', 'nome', 'endereço', 'cidade'])

df_alunos['ra'] = [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009]
df_alunos['nome'] = ['pedro', 'maria', 'jose', 'joao', 'rita', 'julia', 'amanda', 'tiago', 'mateus', 'luis']

# tabela_disciplinas → colunas = (cod_disc, nome_disc, carga_hor)

df_disciplinas = pd.DataFrame(columns = ['cod_disc', 'nome_disc', 'carga_hor'])

df_disciplinas['cod_disc'] = [100, 101, 102, 103]

df_disciplinas['nome_disc'] = ['banco de dados', 'fundamentos da programacao', 'matematica', 'estatistica']

df_disciplinas['carga_hor'] = [80, 60, 60, 60]

# tabela_professores → colunas = (cod_prof, nome, endereco, cidade)

df_professores = pd.DataFrame(columns = ['cod_prof', 'nome', 'endereco', 'cidade'])

# tabela_turma → colunas = (cod_disc, cod_turma, cod_prof, ano, semestre)

df_turma = pd.DataFrame(columns = ['cod_disc', 'cod_turma', 'cod_prof', 'ano', 'semestre'])

# tabela_historico, colunas = (ra, cod_disc, cod_turma, cod_prof, ano, semestre, frequencia, nota)

df_historico = pd.DataFrame(columns = ['ra', 'cod_disc', 'cod_turma', 'cod_prof', 'ano', 'semestre', 'frequencia', 'nota'])

df_historico['ra'] = np.random.randint(1000, 1010, size = 40)

df_historico['cod_disc'] = np.random.randint(100, 104, size = 40)

df_historico['ano'] = np.random.randint(2018, 2021, size = 40)

df_historico['nota'] = np.random.randint(0, 11, size = 40)


#### Criar um banco de dados e fazer a conexão

In [9]:
# Conectar o banco de dados
db_case_1 = create_engine( 'sqlite:///db_case_1.sqlite', echo=False )
conn = db_case_1.connect()

# Criar as tabelas

# Tabela alunos
schema_alunos = """
    CREATE TABLE alunos (
    ra          TEXT,
    nome        TEXT,
    endereço    TEXT,
    cidade      TEXT
    
)
"""

df_alunos.to_sql( 'alunos', con=conn, if_exists='append', index=False )

# Tabela disciplinas

schema_disciplinas = """
    CREATE TABLE disciplinas (
    cod_disc     INTEGER,
    nome_disc    TEXT,
    carga_hor    INTEGER
    
)
"""

df_disciplinas.to_sql( 'disciplinas', con=conn, if_exists='append', index=False )

# Tabela historico

schema_historico = """
    CREATE TABLE historico (
    ra            TEXT,
    cod_disc      INTEGER,
    cod_turma     TEXT,
    cod_prof      TEXT,
    ano           INTEGER,
    semestre      TEXT,
    frequencia    TEXT,
    nota          INTEGER
    
)
"""

df_historico.to_sql( 'historico', con=conn, if_exists='append', index=False )

#### Queries

In [17]:
# a) Quantos alunos cursaram a disciplina de Banco de Dados em 2019 e 2020?

query = """
        select 
        count (ra) as total_de_alunos
        from historico h 
        where cod_disc = 100 and (ano = 2019 or ano = 2020);
        """
df = pd.read_sql_query( query, conn )
df

,total_de_alunos
0,8


In [18]:
# b) Quais os nomes dos alunos que não foram reprovados em nenhuma disciplina (considere nota mínima igual a 6 para aprovação)?
# Em alguns casos, usar MINUS em vez de EXCEPT.

query1 = """
        select distinct nome
        from alunos a left join historico h on (a.ra = h.ra)
        where nota >= 6
        EXCEPT 
        select distinct nome
        from alunos a left join historico h on (a.ra = h.ra)
        where nota < 6;
        """
df1 = pd.read_sql_query( query1, conn )
df1        

,nome
0,pedro


In [19]:
conn.close()